# Summary

* This is a tutorial on using Python for accessing the Tahoe-100M dataset hosted by the Arc Institute.
* The data can be streamed or downloaded locally.
  * For small jobs (e.g., summarizing the some metadata), streaming is recommended.
  * For large jobs (e.g., training a model), downloading is recommended.
* See the [README](README.md#obs-cell-metadata) for a description of the obs metadata.

# Setup

### Installation

If needed, install the necessary dependencies.

You can use the [conda environment](../conda_envs/python.yml) provided in this git repository.

### Load dependencies

In [ ]:
import io
import pandas as pd
import scanpy as sc
import pyarrow.dataset as ds
import gcsfs

In [ ]:
# initialize GCS file system for reading data from GCS
fs = gcsfs.GCSFileSystem()

### Data location

In [ ]:
# GCS bucket path
gcp_base_path = "gs://arc-ctc-tahoe100/2025-02-25/"

# Obs metadata

* `obs` ≃ cell

### Per-sample

* Useful for quickly summarizing the per-sample metadata (a small file versus the entire obs metadata file; see below).

In [ ]:
# path to sample metadata
infile = "/".join([gcp_base_path.rstrip("/"), 'metadata', 'sample_metadata.parquet'])
infile

In [ ]:
# read just the first 3 rows
sample_metadata = ds.dataset(infile, filesystem=fs, format="parquet").head(3).to_pandas()
sample_metadata

In [ ]:
# select certain columns and row filtering
columns_to_read = ['sample', 'plate', 'mean_gene_count']  # Specify the columns you need
dataset = ds.dataset(infile, filesystem=fs, format="parquet")
sample_metadata = dataset.to_table(filter=(ds.field('mean_gene_count') > 2000), columns=columns_to_read).to_pandas()
sample_metadata 

In [ ]:
# get the number of samples
columns_to_read = ["sample"]  # Specify the columns you need
dataset = ds.dataset(infile, filesystem=fs, format="parquet")
sample_count = dataset.to_table(columns=columns_to_read).to_pandas()["sample"].nunique()
print(f"Number of samples: {sample_count}")

In [ ]:
# get samples per plate
columns_to_read = ["plate", "sample"]  # Specify the columns you need
dataset = ds.dataset(infile, filesystem=fs, format="parquet")
samples_per_plate = dataset.to_table(columns=columns_to_read).to_pandas().groupby("plate").size()
samples_per_plate

### Per-observation

* `obs` ~= cells
* For the sake of this tutorial, we will just pull the first 100000 observations.

In [ ]:
# set the path to the obs_metadata file
infile = "/".join([gcp_base_path.rstrip("/"), 'metadata', 'obs_metadata.parquet'])
infile

In [ ]:
# read a subset of the metadata
obs_metadata = ds.dataset(infile, filesystem=fs, format="parquet").head(100000).to_pandas()
obs_metadata

In [ ]:
# sample count
obs_metadata["sample"].nunique()

In [ ]:
# gene count distribution
pd.options.display.float_format = '{:.0f}'.format
obs_metadata["gene_count"].describe()

In [ ]:
# tscp (UMI) count distribution
pd.options.display.float_format = '{:.0f}'.format
obs_metadata["tscp_count"].describe()

# Reading in h5ad files

* For this tutorial, we will be reading in a subsampled version of 1 h5ad file, since the per-plate h5ad files are rather large. 

In [ ]:
# set the path to the plate metadata file
infile = "gs://arc-ctc-tahoe100/2025-02-25/tutorial/plate3_2k-obs.h5ad"

In [ ]:
# read in the h5ad file
with fs.open(infile, 'rb') as f:
    adata = sc.read_h5ad(f)
adata

In [ ]:
# look at the obs metadata
print(adata.obs.shape)
adata.obs.head()

#### Next steps

You can then use the anndata object for various downsteam analyses

# Downloading files

You can use [gsutil](https://cloud.google.com/storage/docs/gsutil) to download any of the files in the bucket
and work with them locally. 

Please be considerate to the [cost of egress](https://cloud.google.com/storage/pricing) when download the data from Google Cloud Storage.

For example:

```bash
gsutil cp gs://arc-ctc-tahoe100/2025-02-25/tutorial/plate3_2k-obs.h5ad .
```

For large data transfers, it is better to use `gsutil rsync`:

```bash
gsutil rsync gs://arc-ctc-tahoe100/2025-02-25/tutorial/ .
```

***

# Session Info

In [ ]:
!conda list